In [4]:
import cv2
import numpy as np
import xml.etree.ElementTree as ET
from collections import defaultdict
from __future__ import absolute_import, division, print_function
import os
from collections import Counter


In [32]:
# Util Function for GT Lidar Depth Generation

def load_velodyne_points(filename):
    """Load 3D point cloud from KITTI file format
    (adapted from https://github.com/hunse/kitti)
    """
    points = np.fromfile(filename, dtype=np.float32).reshape(-1, 4)
    points[:, 3] = 1.0  # homogeneous
    return points


def read_calib_file(path):
    """Read KITTI calibration file
    (from https://github.com/hunse/kitti)
    """
    float_chars = set("0123456789.e+- ")
    data = {}
    with open(path, 'r') as f:
        for line in f.readlines():
            key, value = line.split(':', 1)
            value = value.strip()
            data[key] = value
            if float_chars.issuperset(value):
                # try to cast to float array
                try:
                    data[key] = np.array(list(map(float, value.split(' '))))
                except ValueError:
                    # casting error: data[key] already eq. value, so pass
                    pass

    return data


def sub2ind(matrixSize, rowSub, colSub):
    """Convert row, col matrix subscripts to linear indices
    """
    m, n = matrixSize
    return rowSub * (n-1) + colSub - 1


def generate_depth_map(calib_dir, velo_filename, cam=2, vel_depth=False):
    """Generate a depth map from velodyne data
    """
    # load calibration files
    cam2cam = read_calib_file(os.path.join(calib_dir, 'calib_cam_to_cam.txt'))
    velo2cam = read_calib_file(os.path.join(calib_dir, 'calib_velo_to_cam.txt'))
    velo2cam = np.hstack((velo2cam['R'].reshape(3, 3), velo2cam['T'][..., np.newaxis]))
    velo2cam = np.vstack((velo2cam, np.array([0, 0, 0, 1.0])))

    # get image shape
    im_shape = cam2cam["S_rect_02"][::-1].astype(np.int32)

    # compute projection matrix velodyne->image plane
    R_cam2rect = np.eye(4)
    R_cam2rect[:3, :3] = cam2cam['R_rect_00'].reshape(3, 3)
    P_rect = cam2cam['P_rect_0'+str(cam)].reshape(3, 4)
    P_velo2im = np.dot(np.dot(P_rect, R_cam2rect), velo2cam)

    # load velodyne points and remove all behind image plane (approximation)
    # each row of the velodyne data is forward, left, up, reflectance
    velo = load_velodyne_points(velo_filename)
    velo = velo[velo[:, 0] >= 0, :]

    # project the points to the camera
    velo_pts_im = np.dot(P_velo2im, velo.T).T
    velo_pts_im[:, :2] = velo_pts_im[:, :2] / velo_pts_im[:, 2][..., np.newaxis]

    if vel_depth:
        velo_pts_im[:, 2] = velo[:, 0]

    # check if in bounds
    # use minus 1 to get the exact same value as KITTI matlab code
    velo_pts_im[:, 0] = np.round(velo_pts_im[:, 0]) - 1
    velo_pts_im[:, 1] = np.round(velo_pts_im[:, 1]) - 1
    val_inds = (velo_pts_im[:, 0] >= 0) & (velo_pts_im[:, 1] >= 0)
    val_inds = val_inds & (velo_pts_im[:, 0] < im_shape[1]) & (velo_pts_im[:, 1] < im_shape[0])
    velo_pts_im = velo_pts_im[val_inds, :]

    # project to image
    depth = np.zeros((im_shape[:2]))
    depth[velo_pts_im[:, 1].astype(np.int), velo_pts_im[:, 0].astype(np.int)] = velo_pts_im[:, 2]

    # find the duplicate points and choose the closest depth
    inds = sub2ind(depth.shape, velo_pts_im[:, 1], velo_pts_im[:, 0])
    dupe_inds = [item for item, count in Counter(inds).items() if count > 1]
    for dd in dupe_inds:
        pts = np.where(inds == dd)[0]
        x_loc = int(velo_pts_im[pts[0], 0])
        y_loc = int(velo_pts_im[pts[0], 1])
        depth[y_loc, x_loc] = velo_pts_im[pts, 2].min()
    depth[depth < 0] = 0

    return depth


In [18]:
# Read XML File from Results, we use Frame 6 as examples
key_file = cv2.FileStorage("../Results/6.xml", cv2.FILE_STORAGE_READ)
map_file = cv2.FileStorage("../Results_mappoint/6.xml", cv2.FILE_STORAGE_READ)
sysmap_file = cv2.FileStorage("../Results_map_system/6.xml", cv2.FILE_STORAGE_READ)

# for some reason __getattr__ doesn't work for FileStorage object in python
# however in the C++ documentation, getNode, which is also available, 
# does the same thing
# note we also have to specify the type to retrieve other wise we only get a 
# FileNode object back instead of a matrix

key_depth = np.asarray(key_file.getNode("Depth").mat())
map_depth = np.asarray(map_file.getNode("Depth").mat())
sysmap_depth = np.asarray(sysmap_file .getNode("Depth").mat()) 

key_file.release()
map_file.release()
sysmap_file.release()

print(np.max(key_depth))
print(np.max(sysmap_depth))
print(np.max(map_depth))


9321.936
9322.115
3657.1462


In [28]:
N = 0
# Results of Keypoint-based anchors 
for i in range(375):
    for j in range(1241):
        if key_depth[i][j] != 0.0:
            N+=1
            print((i,j), key_depth[i][j])
print("Number of Anchors: ", N)

(21, 294) 29.66002
(24, 1023) 15.958005
(26, 297) 34.402573
(26, 991) 16.494413
(26, 1088) 13.5643215
(27, 1085) 13.54389
(29, 249) 29.71771
(30, 66) 18.592777
(30, 67) 18.215313
(30, 1052) 13.770462
(31, 796) 34.220463
(34, 748) 38.559937
(34, 769) 41.50423
(37, 391) 61.0035
(38, 748) 39.54229
(38, 1000) 16.116573
(38, 1105) 12.118705
(39, 94) 18.401524
(39, 748) 42.171738
(39, 1002) 16.002213
(39, 1103) 12.524144
(40, 95) 18.517591
(40, 760) 46.95998
(40, 848) 36.86378
(41, 266) 29.521402
(41, 684) 21.827852
(41, 1036) 15.136981
(42, 684) 23.448854
(43, 724) 21.739573
(43, 1173) 10.491221
(44, 421) 44.870373
(44, 468) 54.0094
(44, 808) 35.381996
(44, 1178) 10.789204
(45, 468) 55.04746
(45, 808) 34.584938
(46, 420) 45.89663
(46, 783) 36.361458
(46, 803) 37.187702
(47, 169) 28.143532
(47, 192) 29.618437
(47, 421) 46.063477
(47, 467) 52.23877
(47, 922) 21.584644
(47, 923) 20.752794
(48, 170) 28.883558
(48, 171) 29.61702
(48, 924) 19.958431
(49, 432) 39.08845
(49, 434) 46.290367
(49, 923

(306, 1203) 8.257183
(307, 76) 8.8605995
(307, 1083) 7.8598237
(308, 1082) 7.9797244
(312, 927) 8.39543
(312, 1036) 7.9186583
(313, 791) 7.9914875
(313, 926) 8.485497
(313, 928) 8.317104
(313, 946) 8.365207
(314, 906) 8.311453
(316, 820) 8.02799
(316, 851) 7.707521
(317, 991) 7.6918225
(318, 851) 7.861079
(319, 77) 7.037519
(319, 240) 7.695149
(322, 331) 8.636242
(322, 1121) 7.0192194
(322, 1122) 7.123983
(323, 821) 7.6195526
(323, 1119) 7.277035
(323, 1121) 7.1166353
(323, 1129) 7.061287
(325, 950) 7.5972395
(326, 950) 7.6575766
(326, 964) 7.5846405
(328, 783) 7.1185265
(329, 828) 7.395638
(330, 1171) 6.964346
(331, 218) 7.3614497
(331, 219) 7.294086
(331, 491) 7.2587585
(331, 492) 7.283276
(331, 493) 7.277081
(331, 494) 7.251864
(331, 588) 7.4317064
(331, 1107) 6.8956265
(332, 215) 7.013785
(332, 495) 7.294461
(332, 1094) 7.0035458
(332, 1108) 6.871027
(333, 580) 1.5899096
(333, 860) 7.0523596
(333, 874) 6.9734573
(334, 874) 7.089154
(335, 685) 7.307005
(336, 687) 7.1332526
(336, 799

In [33]:
N = 0
# Results of Mappoint-based anchors from System.cc
for i in range(375):
    for j in range(1241):
        if sysmap_depth[i][j] != 0.0:
            N+=1
            print((i,j), sysmap_depth[i][j])
print("Number of Anchors: ", N)

(28, 249) 30.368855
(33, 797) 33.537487
(34, 372) 40.552822
(40, 762) 41.96452
(41, 683) 22.414505
(41, 684) 22.078896
(42, 266) 28.511791
(42, 967) 16.032333
(43, 266) 28.5881
(43, 467) 53.679993
(44, 467) 55.556576
(44, 468) 56.96697
(44, 723) 23.163376
(44, 724) 22.290863
(44, 808) 34.555126
(45, 420) 45.50509
(45, 808) 34.899647
(45, 1167) 10.802842
(46, 1164) 10.924005
(49, 433) 47.53775
(51, 1081) 13.344153
(52, 1076) 13.574025
(53, 439) 50.158585
(53, 629) 31.966763
(53, 1029) 15.175582
(54, 437) 50.699787
(54, 629) 32.10435
(55, 315) 29.10907
(56, 317) 31.404762
(56, 514) 34.796013
(56, 1095) 13.197295
(56, 1135) 11.791007
(57, 510) 41.90133
(57, 629) 41.62604
(57, 991) 16.120407
(58, 1014) 15.930045
(59, 605) 43.01155
(59, 988) 16.506905
(60, 786) 40.314857
(61, 191) 30.186798
(61, 498) 56.20061
(62, 807) 36.045635
(63, 191) 30.078705
(63, 782) 43.60982
(67, 393) 40.929424
(68, 556) 67.28174
(68, 557) 76.38427
(69, 557) 74.75033
(69, 757) 43.914574
(69, 758) 45.424316
(70, 396

In [34]:
N = 0
# Results of Mappoint-based anchors from Tracking.cc

for i in range(375):
    for j in range(1241):
        if map_depth[i][j] != 0.0:
            N+=1
            print((i,j), map_depth[i][j])
print("Number of Anchors: ", N)

(2, 1105) 13.052641
(3, 1103) 13.314764
(6, 245) 29.175158
(7, 333) 35.817154
(7, 751) 41.03768
(8, 751) 40.67173
(8, 1080) 13.584948
(9, 825) 32.976517
(10, 135) 26.034712
(10, 138) 26.7278
(10, 751) 41.06637
(10, 1074) 14.420137
(12, 304) 36.70153
(12, 305) 37.809868
(12, 1038) 14.459026
(14, 807) 34.36299
(15, 311) 45.546715
(16, 274) 27.972315
(16, 312) 48.237587
(19, 994) 16.003098
(20, 233) 29.923319
(20, 1114) 12.664814
(20, 1117) 12.16416
(20, 1120) 12.087404
(21, 1117) 12.176567
(22, 235) 30.957678
(22, 1184) 9.376644
(23, 337) 39.74302
(24, 275) 30.360167
(24, 771) 42.178215
(25, 1091) 13.052748
(26, 1089) 13.220329
(26, 1235) 10.651279
(27, 338) 39.550766
(28, 249) 30.370062
(28, 467) 24.30086
(28, 1170) 11.063713
(29, 1053) 14.155838
(29, 1055) 13.927652
(29, 1057) 13.288883
(29, 1158) 11.908112
(29, 1165) 11.576403
(31, 1236) 10.576998
(32, 381) 32.730812
(32, 1045) 15.10548
(32, 1232) 10.748388
(33, 795) 35.267273
(33, 1146) 11.197128
(33, 1147) 11.913982
(34, 372) 40.554

(339, 189) 6.8185015
(340, 549) 7.018774
(341, 1125) 6.6853113
(341, 1172) 6.294819
(342, 1203) 6.411207
(343, 785) 6.307227
(344, 1208) 6.3117466
(345, 1207) 6.374031
(345, 1208) 6.324913
(346, 629) 6.6653357
(347, 495) 5.7025957
(347, 1163) 6.235199
(348, 173) 6.300702
(348, 617) 6.7725773
(348, 630) 6.640416
(348, 855) 6.266423
(349, 618) 6.5958357
(350, 854) 6.236801
(350, 857) 6.143653
(350, 909) 6.220001
(350, 987) 6.632987
(350, 990) 6.472026
(351, 169) 6.5579576
(351, 909) 6.252286
(351, 978) 6.488941
(352, 454) 6.248747
(352, 610) 6.3604345
(352, 906) 6.204262
(354, 65) 5.8646107
(355, 1071) 6.231269
(355, 1072) 6.2154226
(358, 147) 5.956397
(358, 837) 5.9617577
(358, 882) 6.075757
(359, 148) 5.9515495
(361, 1040) 6.211137
(363, 1065) 6.0492153
(364, 126) 6.16912
(365, 125) 6.188223
(365, 126) 6.1380796
(365, 127) 5.6907005
(365, 1012) 6.042242
(366, 1014) 6.033519
(367, 906) 5.927698
(367, 908) 5.8733873
(367, 909) 5.817545
(370, 889) 5.833108
(370, 1231) 5.5020695
(371, 865)

In [35]:
# Lidar GT, load from 6-th frames

calib_dir = "KITTI_Depth_Demo/calib/"
velo_filename = "KITTI_Depth_Demo/2011_09_26/2011_09_26_drive_0001_sync/velodyne_points/data/0000000006.bin"

generate_depth_map(calib_dir, velo_filename, cam=2, vel_depth=False)
N = 0
test_depth = generate_depth_map(calib_dir, velo_filename, cam=2, vel_depth=False)
print(np.max(test_depth))
for i in range(375):
    for j in range(1242):
        if test_depth[i][j] != 0.0:
            N+=1
            print((i,j), test_depth[i][j])
print("number of anchors", N)

78.91442348618034
(121, 1232) 10.830154461345627
(121, 1236) 10.782143932902352
(121, 1241) 10.730135239556622
(122, 1214) 11.02118892528314
(122, 1226) 10.949178508000864
(122, 1230) 10.937183436592225
(123, 1204) 11.461296580365552
(123, 1207) 11.07518450396432
(123, 1210) 11.283260186047714
(124, 1193) 11.567313252775605
(124, 1196) 11.539310178061852
(124, 1200) 11.483300085818563
(127, 1235) 10.818191250871617
(127, 1239) 10.752173595019205
(128, 1213) 10.982205357187715
(128, 1223) 10.924196601559375
(128, 1227) 10.734166499408616
(129, 1201) 11.138224220674514
(129, 1205) 11.268259547388801
(129, 1209) 11.0002105733226
(129, 1216) 11.371293935972314
(129, 1220) 11.274269996246
(130, 1116) 14.990203598552826
(130, 1119) 14.946189950101083
(130, 1197) 11.512310488472034
(130, 1210) 19.150632575838227
(130, 1214) 18.882555817580283
(131, 1090) 15.121197852129352
(131, 1094) 15.076195383040956
(131, 1097) 15.021184479908188
(131, 1109) 15.086217977169307
(131, 1112) 15.0362054637433

(158, 956) 36.49906963797616
(158, 959) 36.0760374053705
(158, 962) 35.69000372616211
(158, 963) 35.11796448394153
(158, 966) 34.869943584938035
(158, 1060) 26.857575754560077
(158, 1064) 26.61957071108871
(158, 1067) 26.263572988262315
(158, 1070) 26.01755950516725
(158, 1073) 25.818561884328815
(158, 1077) 25.619552029760282
(158, 1078) 25.457546945754363
(158, 1082) 25.242540741384115
(159, 239) 32.78490549882809
(159, 242) 32.9789225225866
(159, 245) 33.18694979349558
(159, 248) 33.36297532776056
(159, 251) 33.58600560539119
(159, 254) 33.78702872935911
(159, 255) 33.950054812683774
(159, 258) 34.165081546262506
(159, 261) 34.37610638351822
(159, 264) 34.574134387649025
(159, 266) 34.766147312437916
(159, 273) 35.2792122344982
(159, 276) 35.25817453751247
(159, 279) 35.42319033312862
(159, 282) 37.3786802120192
(159, 284) 37.6017041877964
(159, 287) 37.829728958336105
(159, 290) 38.073763018070615
(159, 291) 38.28680369187035
(159, 294) 38.52882801018371
(159, 297) 38.7778626163238

(183, 104) 24.010101244605295
(183, 109) 24.01809231489069
(183, 112) 24.099069601221156
(183, 116) 24.111065718008664
(183, 119) 24.06706219397653
(183, 122) 23.915064604854827
(183, 126) 23.89305102576537
(183, 129) 23.88104654416356
(183, 130) 23.84303856337173
(183, 176) 26.672711354142354
(183, 178) 24.994822592800105
(183, 182) 16.20826511567794
(183, 185) 16.21425519142226
(183, 187) 16.198262204059297
(183, 188) 26.346690657886878
(183, 190) 16.20425418704809
(183, 191) 26.455679770967798
(183, 193) 16.18925446010954
(183, 194) 25.756726364940288
(183, 195) 25.859711426917062
(183, 196) 16.183255890338717
(183, 199) 16.191248821489015
(183, 202) 16.19824044929269
(183, 205) 16.199241964308882
(183, 206) 16.169236697751373
(183, 209) 16.16423955574284
(183, 212) 16.2242209409736
(183, 259) 16.771083999648276
(183, 262) 16.82406517583389
(183, 265) 16.943046587711518
(183, 266) 17.06102200742695
(183, 269) 17.20599389276796
(183, 272) 17.32697575526303
(183, 275) 17.4559466472841

(205, 701) 35.912245611407606
(205, 703) 35.9262196601517
(205, 706) 35.75329048944172
(205, 708) 35.92417787327131
(205, 710) 35.93614280237609
(205, 713) 35.948118057669994
(205, 715) 35.981081250961374
(205, 716) 35.96107504531769
(205, 718) 35.88309467124352
(205, 721) 35.85809484684737
(205, 723) 35.815094699104236
(205, 725) 35.25537138221467
(205, 728) 34.8155900108531
(205, 730) 33.562253839489486
(205, 731) 35.53415729124351
(205, 734) 35.47216824647924
(205, 736) 35.31323874760275
(205, 738) 35.166288479939695
(205, 741) 34.67854273820083
(205, 743) 34.23976200781026
(205, 745) 33.98287262699568
(205, 747) 33.85093871913413
(205, 749) 33.70099789413137
(205, 751) 33.762937681232096
(205, 754) 33.76391282097826
(205, 756) 33.800867210088555
(205, 759) 33.77186304410565
(205, 761) 33.73585277797265
(205, 762) 33.08620360156686
(205, 765) 32.82332572387814
(205, 767) 32.086717696330666
(205, 772) 34.33240663512361
(205, 774) 34.15448284685167
(205, 776) 34.09149121333743
(205, 7

(224, 696) 22.282660945237037
(224, 698) 22.32960597542062
(224, 699) 22.40054529789081
(224, 701) 22.460468954055134
(224, 704) 22.11375590967614
(224, 706) 22.351532522519836
(224, 709) 22.3285348064363
(224, 711) 22.61727484848796
(224, 713) 22.301528769991364
(224, 714) 22.323503082359146
(224, 717) 21.770950801057076
(224, 721) 22.811031602949413
(224, 724) 22.810012552002927
(224, 726) 22.67211752169534
(224, 728) 22.861937976174055
(224, 730) 22.914876600982282
(224, 732) 22.76099455330883
(224, 734) 22.815929151668172
(224, 737) 22.846886292686147
(224, 739) 22.898820795524205
(224, 741) 22.86482412615616
(224, 744) 22.831837303495067
(224, 745) 22.691948579949926
(224, 747) 22.85877989952727
(224, 750) 22.854771080494523
(224, 752) 22.784809303686334
(224, 754) 22.640914828474365
(224, 757) 22.70882780537726
(224, 759) 22.602899136994917
(224, 762) 22.2711684028354
(224, 763) 22.227198052075337
(224, 765) 22.027350550614912
(224, 768) 21.62267859263457
(224, 775) 22.4488935612

...

(271, 688) 11.990025569232499
(271, 690) 12.008976726494557
(271, 692) 11.992994108331091
(271, 695) 11.988989766423977
(271, 697) 11.982985803061506
(271, 699) 12.003948111563728
(271, 702) 12.008921122371701
(271, 703) 12.02489735659818
(271, 705) 12.094772419998382
(271, 707) 12.116724377706054
(271, 710) 12.131686495106408
(271, 712) 12.139660430716361
(271, 714) 12.118677969914698
(271, 717) 11.966913410139849
(271, 718) 11.731283321979713
(271, 733) 11.558487494820936
(271, 734) 11.653320497946796
(271, 736) 11.642327269354597
(271, 739) 11.587399520025164
(271, 741) 11.577406765528547
(271, 743) 11.571401848558667
(271, 746) 11.543430748568412
(271, 748) 11.526437453245535
(271, 749) 11.488490185744652
(271, 752) 11.51443118455992
(271, 754) 11.476481554139859
(271, 757) 11.515401055825393
(271, 759) 11.50639712128318
(271, 761) 11.621194902273178
(271, 764) 11.726003575439732
(271, 766) 11.65210974113264
(271, 767) 11.71999712475724
(271, 770) 11.67404875974373
(271, 782) 11.56

(292, 9) 12.195721061476949
(292, 73) 12.062580329239985
(292, 75) 12.059607483760777
(292, 79) 12.071621596386516
(292, 134) 11.499201450248393
(292, 136) 11.501218989265498
(292, 139) 11.515234673199062
(292, 142) 11.521259169624791
(292, 174) 11.175544646544138
(292, 175) 11.18455206841726
(292, 179) 11.185576847881176
(292, 182) 11.179623275504953
(292, 238) 10.742970170509308
(292, 241) 10.7380064142436
(292, 244) 10.739032769170876
(292, 247) 10.74204854492216
(292, 248) 10.738066141609425
(292, 360) 10.05066273965719
(292, 361) 10.056660940908971
(292, 363) 10.078634589684887
(292, 366) 10.080650760693027
(292, 369) 10.083656955236428
(292, 371) 10.116610230393679
(292, 374) 10.104647633121434
(292, 376) 10.126622235519584
(292, 513) 9.940349696906395
(292, 515) 9.915398393700054
(292, 517) 9.92538364622417
(292, 519) 9.944349566758897
(292, 521) 9.923388257036494
(292, 523) 9.943351583838115
(292, 526) 9.962325966314141
(292, 528) 9.95034416527885
(292, 530) 9.966318327847576
(

(316, 925) 8.36703098422559
(316, 928) 8.352039632004091
(316, 930) 8.346025382509408
(316, 932) 8.338024782451889
(316, 935) 8.332010409846736
(316, 937) 8.323997492504107
(316, 940) 8.298038083772651
(316, 1035) 7.974580397996168
(316, 1039) 7.92466580229884
(316, 1040) 7.894719964121348
(316, 1043) 7.866750730841055
(316, 1047) 7.834781299752277
(316, 1050) 7.798834731759476
(316, 1123) 7.531317240826238
(316, 1127) 7.514315462888421
(316, 1130) 7.557168783854202
(316, 1213) 7.528107528038575
(316, 1216) 7.7086142852996
(317, 59) 9.244541501789852
(317, 63) 9.255557711954307
(317, 66) 9.266572967309937
(317, 138) 8.89992925784419
(317, 142) 8.898965489411136
(317, 145) 8.911971739808239
(317, 148) 8.914997510026842
(317, 213) 8.604291263167067
(317, 216) 8.616297411365917
(317, 217) 8.614316206173314
(317, 220) 8.610353793296317
(317, 278) 8.389538427804602
(317, 281) 8.409524356762823
(317, 284) 8.414530918973893
(317, 287) 8.448484460757498
(317, 365) 8.360350500134773
(317, 368) 

(342, 914) 6.53344558803349
(342, 915) 6.460629283240966
(342, 918) 6.408746543282544
(342, 921) 6.303005455870503
(342, 924) 6.237165741166422
(342, 927) 6.192261915389774
(342, 930) 6.1304011424270035
(342, 982) 7.008162828676334
(342, 985) 7.006138821407971
(342, 988) 6.974191268197839
(342, 991) 6.954221370678476
(342, 992) 6.955198475790799
(342, 1062) 6.799511346089677
(342, 1064) 6.758608876184296
(342, 1067) 6.710705576697839
(342, 1071) 6.668780113623484
(342, 1074) 6.623865732372728
(342, 1155) 6.408307455554756
(342, 1157) 6.377372589998115
(342, 1161) 6.385304615961219
(343, 24) 7.687630253559105
(343, 26) 7.697628443919083
(343, 30) 7.710634464971904
(343, 88) 7.464986450680152
(343, 92) 7.4749929774088235
(343, 95) 7.493989132760756
(343, 170) 7.125553222622606
(343, 174) 7.132570354304457
(343, 177) 7.1375871402381765
(343, 245) 7.1413675437320805
(343, 248) 7.167322350951889
(343, 249) 7.17432057014695
(343, 308) 7.086321235663276
(343, 311) 7.099307349319088
(343, 314)

(370, 1196) 5.778513594408583
(370, 1200) 5.773489617191542
(370, 1204) 5.732565250039064
(371, 12) 6.390001433689839
(371, 16) 6.404008073853422
(371, 62) 6.291130316638439
(371, 64) 6.287159463710029
(371, 68) 6.2981664851337955
(371, 118) 6.130451557483213
(371, 122) 6.150427554024694
(371, 125) 6.1604349524938105
(371, 174) 6.147309760057255
(371, 177) 6.163295991153884
(371, 243) 6.0554234307054156
(371, 246) 6.075389101057768
(371, 249) 6.085386296507529
(371, 314) 6.019359961953267
(371, 317) 6.024367166649453
(371, 320) 6.042332965543354
(371, 323) 6.048340643074401
(371, 409) 6.070041081761918
(371, 411) 6.073049141799042
(371, 413) 6.061089776783591
(371, 415) 6.065087133571412
(371, 418) 6.063105078468946
(371, 504) 5.97112257764877
(371, 507) 5.972119963256846
(371, 509) 5.981096506574341
(371, 511) 5.976125407648741
(371, 514) 5.98410160248774
(371, 516) 5.9811203987796056
(371, 519) 5.986106825487865
(371, 520) 5.989105801749978
(371, 522) 5.999071290624134
(371, 525) 5.9